In [20]:
import pandas as pd
import calendar
import requests
import numpy as np
from datetime import datetime
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [45]:
def get_df(url: str) -> pd.DataFrame:
    """Given a url, scrapes the data and returns a pandas dataframe"""

    html = requests.get(url)
    soup = BeautifulSoup(html.text, "html.parser")

    tr_parent1 = soup.find_all("tbody", {"class": "table__tbody"})
    headers = [
        "isin",
        "name",
        "currency",
        "coupon",
        "yield",
        "moody's rating",
        "maturity date",
        "bid",
        "ask",
    ]
    data_dict = {header: [] for header in headers}

    # Extract relevant data from table cells
    cell = 0
    table_cells = tr_parent1[0].find_all("td")
    table_cell_count = len(table_cells)
    storage = []
    for cell in range(table_cell_count):
        if cell % 8 == 0:
            isin = str(table_cells[cell]).split("-")[-1].split('"')[0]
            storage.append(isin)
        data = tr_parent1[0].find_all("td", {"class": "table__td"})
        extracted_data = data[cell].text.strip()
        storage.append(extracted_data)

    for i in range(len(storage)):
        data_dict[headers[i % 9]].append(storage[i])

    # Convert to pandas dataframe
    df = pd.DataFrame(data_dict)
    df[["bid", "ask"]] = df[["bid", "ask"]].replace("-", "NaN").astype(float)
    df = df.drop(columns=["name", "currency", "moody's rating"])
    df["close"] = (df["bid"] + df["ask"]) / 2

    storage = []
    for i in range(df.shape[0]):
        coupon = df["coupon"].values[i].split("%")[0][:3]
        df["maturity date"] = pd.to_datetime(df["maturity date"])

        month = calendar.month_name[df.iloc[i]["maturity date"].month][:3]
        year = df.iloc[i]["maturity date"].year % 100
        new_col = f"CAN {coupon} {month} {year}"
        storage.append(new_col)

    df = df.drop(columns=["bid", "ask"])
    df["name"] = np.array(storage)
    df = df.set_index("name")

    return df


from typing import List


def get_all_daily_df(urls: List[str]) -> None:
    """Given a list of urls, generate the JSON files"""

    dataframes = {}

    for i in range(3):
        dataframes[f"df{i}"] = get_df(urls[i])

    today = datetime.now()
    filename = f'{today.strftime("%d")}{today.strftime("%b")}'
    dfs = [dataframes["df0"], dataframes["df1"], dataframes["df2"]]
    concatenated_dfs = pd.concat(dfs).drop_duplicates()
    print(concatenated_dfs)
    concatenated_dfs.reset_index().to_json(f"data/{filename}.json")

In [3]:
urls = [
    "https://markets.businessinsider.com/bonds/finder?borrower=71&maturity=shortterm&yield=&bondtype=2%2c3%2c4%2c16&coupon=&currency=184&rating=&country=19",
    "https://markets.businessinsider.com/bonds/finder?p=2&borrower=71&maturity=shortterm&yield=&bondtype=2%2c3%2c4%2c16&coupon=&currency=184&rating=&country=19",
    "https://markets.businessinsider.com/bonds/finder?borrower=71&maturity=midterm&yield=&bondtype=2%2c3%2c4%2c16&coupon=&currency=184&rating=&country=19",
]

#### Generate 10 JSON files corresponding to each of the 10 weekdays between 8 Jan - 22 Jan.

In [46]:
get_all_daily_df(urls)

                        isin   coupon  yield maturity date    close
name                                                               
CAN 3.0 Nov 24  ca135087p402  3.0000%  4.89%    2024-11-01   98.655
CAN 2.2 Jun 25  ca135087d507  2.2500%  4.43%    2025-06-01   97.265
CAN 1.5 Jun 26  ca135087e679  1.5000%  3.91%    2026-06-01   94.710
CAN 0.7 Feb 24  ca135087m920  0.7500%      -    2024-02-01   99.995
CAN 3.7 May 25  ca135087q319  3.7500%  4.50%    2025-05-01   99.130
CAN 1.5 Apr 25  ca135087n340  1.5000%  4.54%    2025-04-01   96.630
CAN 4.5 Feb 26  ca135087r226  4.5000%  4.05%    2026-02-01  100.820
CAN 0.2 Apr 24  ca135087l690  0.2500%  4.80%    2024-04-01   99.075
CAN 1.0 Sep 26  ca135087l930  1.0000%  3.83%    2026-09-01   93.130
CAN 3.0 Oct 25  ca135087p246  3.0000%  4.26%    2025-10-01   97.970
CAN 2.7 Aug 24  ca135087n910  2.7500%  5.01%    2024-08-01   98.945
CAN 3.5 Aug 25  ca135087q640  3.5000%  4.30%    2025-08-01   98.835
CAN 4.5 Nov 25  ca135087q806  4.5000%  4.23%    

#### Extract out only the 10 selected bonds of interest. They are:

```
1. CAN 0.7 Oct 24
2. CAN 1.5 Sep 24
3. CAN 1.0 Sep 26
4. CAN 1.5 Apr 25
5. CAN 1.2 Mar 25
6. CAN 1.5 Jun 26
7. CAN 1.5 May 24
8. CAN 0.5 Sep 25
9. CAN 1.0 Jun 27
10. CAN 1.2 Mar 27
```

In [41]:
# List the ISIN for each of the 10 selected bonds.

BONDS = [
    "ca135087m508",
    "ca135087j967",
    "ca135087l930",
    "ca135087n340",
    "ca135087k528",
    "ca135087e679",
    "ca135087n423",
    "ca135087k940",
    "ca135087f825",
    "ca135087m847",
]

assert len(BONDS) == 10

#### Load the dataframes from the JSON files, and filter to only include the 10 selected bonds. Then save the dataframes back into the JSON files.

In [62]:
dates = [
    "08",
    "09",
    "10",
    "11",
    "12",
    "15",
    "16",
    "17",
    "18",
    "19",
]

for date in dates:
    df = (
        pd.read_json(f"data/{date}Jan.json")
        .set_index(["name"])
        .sort_values(["maturity date"], ascending=[True])
    )

    df = df.loc[df["isin"].isin(BONDS)]

    assert df.shape[0] == 10

    df.reset_index().to_json(f"data/{date}Jan.json")